# Convolutional Neural Network

#### We use tensorFlow to build the Neural Network


In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from keras.utils import to_categorical
import os
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


### Importing Dataset


In [2]:
def ImportValues(namex,namey):
   # This function loads the x and y values, reshapes X to volume and tranform Y to 1-hot 
    X=np.load(namex)
    Y=np.load(namey)
    Y=np.squeeze(Y)

    # length check
    assert(len(X) == len(Y))


    n_classes = 3 # 3 labels: 0:normal, 1:MCI, 2:Alzh

    X = np.reshape(X,(-1,256,256,170,1))

    # choosing a 16 by 16 by 16 slot in the images
    X =X[:, 100:116, 100:116, 100:116, :]

    # change lables to 1-hot form matrix
    Y = to_categorical(Y, n_classes)

    return X ,Y

### Definiton of Convolutional Newral Network

In [51]:
def cnn_model(x_train_data, n_x , n_y , n_z , keep_rate=0.7, seed=None ):

    #Definition of the convolution net
    
    # conv => n_x*n_y*n_z
    conv1 = tf.layers.conv3d(inputs=x_train_data, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => n_x*n_y*n_z
    conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => n_x*n_y*n_z/2*2*2
    pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2)

    # conv => n_x*n_y*n_z/2*2*2
    conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => n_x*n_y*n_z/2*2*2
    conv5 = tf.layers.conv3d(inputs=conv4, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => n_x*n_y*n_z/4*4*4
    pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2)

    # conv => n_x*n_y*n_z/4*4*4
    conv7 = tf.layers.conv3d(inputs=pool6, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # conv => n_x*n_y*n_z/4*4*4
    conv8 = tf.layers.conv3d(inputs=conv7, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu)
    # pool => n_x*n_y*n_z/8*8*8
    pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2)

    cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, training=True)

    flattening = tf.reshape(cnn3d_bn, [-1, ((n_x//2)//2)//2*((n_y//2)//2)//2*((n_z//2)//2)//2*128])
    dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu)
    # (1-keep_rate) is the probability that the node will be kept
    dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=True)

    y_conv = tf.layers.dense(inputs=dropout, units=3)

    return y_conv

### Training Neural Network

In [58]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, learning_rate=0.05, keep_rate=0.7, epochs=20, batch_size=128):

    #dimensions of our input and output
    n_x = 16
    n_y = 16
    n_z = 16
    n_classes = 3
    
    # name of the .txt file that is created
    name='First try'

    f = open('./'+name+'.txt', 'w')
    f.write('First Try ')

    
    x_input = tf.placeholder(tf.float32, shape=[None, 16, 16, 16, 1], name='Input' )
    y_input = tf.placeholder(tf.float32, shape=[None, n_classes], name='Output') 


    prediction = cnn_model(x_input, n_x , n_y , n_z , keep_rate, seed = 1 , name='Prediction' )
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input), name='Cost' )


    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
           
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

    iterations = int(len(x_train_data)/batch_size) + 1

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        import datetime

        start_time = datetime.datetime.now()

        iterations = int(len(x_train_data)/batch_size) + 1
        # run epochs
        for epoch in range(epochs):
            start_time_epoch = datetime.datetime.now()
            print('Epoch', epoch, 'started', end='')
            epoch_loss = 0
            train_acc = 0
            # mini batch
            for itr in range(iterations):
                mini_batch_x = x_train_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y = y_train_data[itr*batch_size: (itr+1)*batch_size]
                _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
                epoch_loss += _cost
                train_acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            #  using mini batch in case not enough memory
            acc = 0
            itrs = int(len(x_test_data)/batch_size) + 1
            for itr in range(itrs):
                mini_batch_x_test = x_test_data[itr*batch_size: (itr+1)*batch_size]
                mini_batch_y_test = y_test_data[itr*batch_size: (itr+1)*batch_size]
                acc += sess.run(accuracy, feed_dict={x_input: mini_batch_x_test, y_input: mini_batch_y_test})
            end_time_epoch = datetime.datetime.now()
            print(' Training Set Accuracy:' + str(train_acc/itrs) +' Testing Set Accuracy:' + str(acc/itrs) + ' Time elapse: ' + str(end_time_epoch - start_time_epoch))
            f.write('\n'+' Training Set Accuracy:' + str(train_acc/itrs) +' Testing Set Accuracy:' + str(acc/itrs) + ' Time elapse: ' + str(end_time_epoch - start_time_epoch)) 
        end_time = datetime.datetime.now()
        print('Time elapse: ', str(end_time - start_time))
        
        
        

### Example of Training

In [59]:
namex='./XValues1.npy'
namey='./YValues1.npy'

x_train ,y_train = ImportValues(namex,namey)

namex='./XValues2.npy'
namey='./YValues2.npy'

x_test ,y_test = ImportValues(namex,namey)


train_neural_network(x_train, y_train, x_test, y_test, epochs=40, batch_size=3, learning_rate=0.0001)

Epoch 0 started Training Set Accuracy:0.75 Testing Set Accuracy:0.5000000074505806 Time elapse: 0:00:02.388812
Epoch 1 started Training Set Accuracy:0.8333333432674408 Testing Set Accuracy:0.2500000074505806 Time elapse: 0:00:02.252788
Epoch 2 started Training Set Accuracy:0.5 Testing Set Accuracy:0.0833333358168602 Time elapse: 0:00:02.310382
Epoch 3 started Training Set Accuracy:0.3333333358168602 Testing Set Accuracy:0.25 Time elapse: 0:00:02.307658
Epoch 4 started Training Set Accuracy:0.6666666716337204 Testing Set Accuracy:0.2500000074505806 Time elapse: 0:00:02.276190
Epoch 5 started Training Set Accuracy:0.3333333432674408 Testing Set Accuracy:0.416666679084301 Time elapse: 0:00:02.265710
Epoch 6 started Training Set Accuracy:0.7500000074505806 Testing Set Accuracy:0.3333333432674408 Time elapse: 0:00:02.262014
Epoch 7 started Training Set Accuracy:0.7500000149011612 Testing Set Accuracy:0.2500000074505806 Time elapse: 0:00:02.281332
Epoch 8 started Training Set Accuracy:0.8333